In [8]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
import cv2
import torch
import torch.nn as nn
import os
import kornia
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" #Prevents an unusual error where displaying images will crash the kernel?
#rcParams['figure.figsize'] = 11,8
#plt.gray() #Ensures greyscale images are displayed gray without having to use cmap parameter
import matplotlib; matplotlib.use('TkAgg')
#matplotlib.use('Qt5Agg')

In [165]:
import glob
images = [plt.imread(file) for file in glob.glob("biker/*.jpg")]

In [166]:
images

[array([[[228, 221, 211],
         [235, 228, 218],
         [245, 238, 228],
         ...,
         [255, 255, 247],
         [255, 255, 247],
         [255, 255, 247]],
 
        [[228, 221, 211],
         [235, 228, 218],
         [245, 238, 228],
         ...,
         [255, 255, 247],
         [255, 255, 247],
         [255, 255, 247]],
 
        [[228, 221, 211],
         [235, 228, 218],
         [245, 238, 228],
         ...,
         [255, 255, 247],
         [255, 255, 247],
         [255, 255, 247]],
 
        ...,
 
        [[192, 151,  99],
         [192, 151,  99],
         [192, 151,  99],
         ...,
         [210, 162, 113],
         [210, 162, 113],
         [210, 162, 113]],
 
        [[192, 151,  99],
         [192, 151,  99],
         [192, 151,  99],
         ...,
         [210, 162, 113],
         [210, 162, 113],
         [210, 162, 113]],
 
        [[192, 151,  99],
         [192, 151,  99],
         [192, 151,  99],
         ...,
         [210, 162, 113],
  

In [ ]:
# fig, ax = plt.subplots(1,len(images), figsize=(50,50))
# for i in range(len(images)):
#     ax[i].imshow(images[i])

In [167]:
images = np.array(images)
images.shape

(19, 720, 1280, 3)

In [ ]:
# image = images[0]
# plt.imshow(image)
# plt.ginput()

In [12]:
points = []
for image in images:
    plt.imshow(image)
    point = plt.ginput()
    points.append(point)

In [168]:
len(points)

19

In [169]:
translation = np.zeros_like(points)
translation

array([[[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]],

       [[0., 0.]]])

In [8]:
points[-1][0][0] #x


540.0278592375366

In [9]:
points[-1][0][1] #y

437.2759530791788

In [170]:
#Array of translations
for i in range(len(points)):
    translation[i][0][0] = points[0][0][0] - points[i][0][0] #x difference
    translation[i][0][1] = points[0][0][1] - points[i][0][1] #y difference


In [171]:
translation = torch.Tensor(translation)
# translation = translation.squeeze(1)

In [172]:
translation.shape

torch.Size([19, 1, 2])

In [173]:
translation

tensor([[[   0.0000,    0.0000]],

        [[   3.7537,  -30.0293]],

        [[  -3.7537,  -52.5513]],

        [[  56.3050,  -71.3196]],

        [[ 133.2551,  -91.9648]],

        [[ 167.0381,  -97.5953]],

        [[ 108.8563, -137.0088]],

        [[ 108.8563, -137.0088]],

        [[  -9.3842, -144.5161]],

        [[ -46.9208, -167.0381]],

        [[-108.8563, -163.2845]],

        [[-125.7478, -121.9941]],

        [[-121.9941,  -52.5513]],

        [[ -91.9648,   84.4575]],

        [[  -1.8768,   -5.6305]],

        [[  -3.7537, -125.7478]],

        [[  -9.3842, -180.1759]],

        [[  50.6745, -170.7918]],

        [[  75.0733, -150.1466]]])

In [174]:
#Converts numpy arrays to Kornia compatable tensors
def toKornia(img):
    print(img.shape)
    img = kornia.image_to_tensor(img) #Kornia in shape BCHW
    print(img.shape)
    #Adding in dummy axis from: https://sparrow.dev/adding-a-dimension-to-a-tensor-in-pytorch/
    #img = torch.unsqueeze(img, dim=0)
    print(img.shape)
    return img

In [175]:
images = images.astype(np.float32)/255 

In [70]:
# images.shape
# for img in images

In [176]:
images_k = kornia.image_to_tensor(images)

In [177]:
images_k.shape

torch.Size([19, 3, 720, 1280])

In [178]:
translation = translation.astype(np.float32)/255 

AttributeError: 'Tensor' object has no attribute 'astype'

In [179]:
translation_k = kornia.image_to_tensor(translation)

TypeError: expected np.ndarray (got Tensor)

In [153]:
translation_k = torch.unsqueeze(translation_k, dim=0)

In [155]:
translation_k = translation_k.squeeze()

In [157]:
translation_k = translation_k.permute(1,0)

In [180]:
translation_k.shape

torch.Size([19, 2])

In [159]:
translation_k

tensor([[ 0.0000,  0.0000],
        [ 0.0147, -0.1178],
        [-0.0147, -0.2061],
        [ 0.2208, -0.2797],
        [ 0.5226, -0.3606],
        [ 0.6551, -0.3827],
        [ 0.4269, -0.5373],
        [ 0.4269, -0.5373],
        [-0.0368, -0.5667],
        [-0.1840, -0.6551],
        [-0.4269, -0.6403],
        [-0.4931, -0.4784],
        [-0.4784, -0.2061],
        [-0.3606,  0.3312],
        [-0.0074, -0.0221],
        [-0.0147, -0.4931],
        [-0.0368, -0.7066],
        [ 0.1987, -0.6698],
        [ 0.2944, -0.5888]])

In [160]:
images_k

tensor([[[[1.3750e-05, 1.4173e-05, 1.4776e-05,  ..., 1.5379e-05,
           1.5379e-05, 1.5379e-05],
          [1.3750e-05, 1.4173e-05, 1.4776e-05,  ..., 1.5379e-05,
           1.5379e-05, 1.5379e-05],
          [1.3750e-05, 1.4173e-05, 1.4776e-05,  ..., 1.5379e-05,
           1.5379e-05, 1.5379e-05],
          ...,
          [1.1579e-05, 1.1579e-05, 1.1579e-05,  ..., 1.2665e-05,
           1.2665e-05, 1.2665e-05],
          [1.1579e-05, 1.1579e-05, 1.1579e-05,  ..., 1.2665e-05,
           1.2665e-05, 1.2665e-05],
          [1.1579e-05, 1.1579e-05, 1.1579e-05,  ..., 1.2665e-05,
           1.2665e-05, 1.2665e-05]],

         [[1.3328e-05, 1.3750e-05, 1.4353e-05,  ..., 1.5379e-05,
           1.5379e-05, 1.5379e-05],
          [1.3328e-05, 1.3750e-05, 1.4353e-05,  ..., 1.5379e-05,
           1.5379e-05, 1.5379e-05],
          [1.3328e-05, 1.3750e-05, 1.4353e-05,  ..., 1.5379e-05,
           1.5379e-05, 1.5379e-05],
          ...,
          [9.1066e-06, 9.1066e-06, 9.1066e-06,  ..., 9.7700

In [183]:
translation.shape

torch.Size([19, 2])

In [182]:
translation = translation.squeeze()

In [188]:
#placeholder = kornia.geometry.transform.translate(images_k, translation_k)
placeholder = kornia.geometry.transform.translate(images_k, translation)


In [189]:
placeholder

tensor([[[[8.9412e-01, 9.2157e-01, 9.6078e-01,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          [8.9412e-01, 9.2157e-01, 9.6078e-01,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          [8.9412e-01, 9.2157e-01, 9.6078e-01,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          ...,
          [7.5294e-01, 7.5294e-01, 7.5294e-01,  ..., 8.2353e-01,
           8.2353e-01, 8.2353e-01],
          [7.5294e-01, 7.5294e-01, 7.5294e-01,  ..., 8.2353e-01,
           8.2353e-01, 8.2353e-01],
          [7.5294e-01, 7.5294e-01, 7.5294e-01,  ..., 8.2353e-01,
           8.2353e-01, 8.2353e-01]],

         [[8.6667e-01, 8.9412e-01, 9.3333e-01,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          [8.6667e-01, 8.9412e-01, 9.3333e-01,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          [8.6667e-01, 8.9412e-01, 9.3333e-01,  ..., 1.0000e+00,
           1.0000e+00, 1.0000e+00],
          ...,
          [5.9216e-01, 5.9216e-01, 5.9216e-01,  ..., 6.3529

In [186]:
placeholder.shape

torch.Size([19, 3, 720, 1280])

In [109]:
#Displays Tensors as image
def imshowTorch(input):
    image = kornia.tensor_to_image(input) #Converts to HWC format
    plt.imshow(image) #As matplotlib requires arrays 

In [33]:
whatt = kornia.tensor_to_image(placeholder[0])

In [34]:
#whatt = whatt.astype(np.uint8)*255 
whatt = whatt * 255

In [35]:
whatt = whatt.astype(np.uint8)

In [36]:
whatt.shape

(720, 1280, 3)

In [37]:
whatt

array([[[228, 221, 211],
        [235, 228, 218],
        [245, 238, 228],
        ...,
        [255, 255, 246],
        [255, 255, 246],
        [255, 255, 246]],

       [[228, 221, 211],
        [235, 228, 218],
        [245, 238, 228],
        ...,
        [255, 255, 247],
        [255, 255, 247],
        [255, 255, 247]],

       [[228, 221, 211],
        [235, 228, 218],
        [245, 238, 228],
        ...,
        [255, 255, 247],
        [255, 255, 247],
        [255, 255, 247]],

       ...,

       [[192, 151,  99],
        [192, 151,  99],
        [192, 151,  99],
        ...,
        [210, 162, 113],
        [210, 162, 113],
        [210, 162, 113]],

       [[192, 151,  99],
        [192, 151,  99],
        [192, 151,  99],
        ...,
        [210, 162, 113],
        [210, 162, 113],
        [210, 162, 113]],

       [[192, 151,  99],
        [192, 151,  99],
        [192, 151,  99],
        ...,
        [210, 162, 113],
        [210, 162, 113],
        [210, 162, 113]]

In [134]:
cv2.imwrite("out_img.jpg",whatt) #As matplotlib requires arrays 

True

In [190]:
cool_result = []
for i in range (0, len(placeholder)):
    result = kornia.tensor_to_image(placeholder[i])
    result = result * 255
    result = result.astype(np.uint8)
    cv2.imwrite("result" + str(i) + ".jpg",result)